# Regularisierung auf dem Titanic-Datensatz

Hinweis: Da das Notebook nur das Prinizip der Regularisierung zeigen soll, ist der ML-Worflow in diesem Notebook stark vereinfacht (d.h. kein Auffüllen von N/As, kein Feature-Scaling und keine kategorischen Features).

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

pd.options.mode.chained_assignment = None  # avoid slide-copy-warning 

In [49]:
df = pd.read_csv("data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [50]:
df_selection = df[["Survived", "Pclass", "Age", "SibSp", "Parch", "Fare"]]
df_selection = df_selection.dropna()
df_selection.head()

,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


In [51]:
df_X = df_selection.drop(columns = ["Survived"])
df_y = df_selection["Survived"]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

## Aufgaben:

### Aufgabe 1: Ridge-Regularisierung
1. Trainieren Sie eine Logistische Regression mit Ridge-Regularisierung (siehe [RidgeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html)) mit $\alpha =100$ auf den Trainingsdaten.
2. Bestimmen Sie die Accuracy auf den Testdaten.

In [52]:
# train logistic regression model with ridge regularization using RidgeClassifier
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier(alpha=100.0)
clf.fit(X_train, y_train)

RidgeClassifier(alpha=100.0)

In [53]:
pred = clf.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, pred.round()))
print("Feature weights:")
print(pd.Series(clf.coef_[0], index=X_train.columns))

# q: why do we need to do clf.coef_[0] here?
# a: clf.coef_ is a 2D array, with one row per class.
#    we only have one class here, so we can just take the first row.

print("Sum of absolute weights: ", np.sum(np.abs(clf.coef_)))

Accuracy score:  0.6713286713286714
Feature weights:
Pclass   -0.320888
Age      -0.013431
SibSp    -0.096943
Parch     0.092842
Fare      0.002719
dtype: float64
Sum of absolute weights:  0.5268232381956689


### Aufgabe 2: Cross-Validation über $\alpha$
1. Führen Sie nun eine Cross-Validation mit Hilfe der Klasse [RidgeClassifierCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifierCV.html) durch. Benutzen Sie dafür die gleichen Werte für $\alpha$ wie im Notebook [`4_OverUnderfitting.ipynb`](4_OverUnderfitting.ipynb) in Zelle 9.
2. Bestimmen Sie die Accuracy auf den Testdaten.

In [54]:
# import RidgeCV to find the best alpha value
from sklearn.linear_model import RidgeCV

# create a list of alphas to try ranging from 0.1 to 10000
alphas = np.logspace(-1, 4, 100)

# q: what is -1 and 4 in np.logspace(-1, 4, 100)?
# a: -1 and 4 are the exponents of 10, so we are trying alphas ranging from 0.1 to 10000


print(f"Testing {len(alphas)} different values for Alpha")

# create and fit a ridge regression model, testing each alpha
model = RidgeCV(alphas=alphas, scoring='neg_mean_squared_error')
model.fit(X_train, y_train)

# summarize the results of the grid search
print("Best Alpha: ", model.alpha_)
print("Best Score: ", model.best_score_)
print("Best Coefficients: ", model.coef_)
print("Sum of absolute weights: ", np.sum(np.abs(model.coef_)))

Testing 100 different values for Alpha
Best Alpha:  6.579332246575679
Best Score:  -0.19603116680069563
Best Coefficients:  [-0.22209922 -0.00789843 -0.0631952   0.06616771  0.00086922]
Sum of absolute weights:  0.3602297691243238


In [55]:
# determine accuracy on the test set
pred = model.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, pred.round()))

# q: why do we need to do pred.round() here?
# a: pred is a continuous value, but we need to round it to 0 or 1 to compare it to y_test

Accuracy score:  0.6853146853146853


### Aufgabe 3:  Lasso-Regularisierung
Für die Lasso-Regularisierung gibt es keine extra Unterklasse, da diese bei der Klasse `LogisitcRegression` über den Parameter `penalty` eingestellt werden kann. Mit dem Parameter `penalty = "l1"` wird die Lasso-Regression verwendet.
Der Regularisierungsfaktor $\alpha$ wird in diesem Fall über den Parameter `C` bestimmt, welcher den Default-Wert `C=1.0` hat:

`LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")`

Beachten Sie dabei:
- Für die Regression ist das zusätzliche Argument `solver="liblinear"` nötig, da der Standard-Optimierer nicht mit `L1` funktioniert.
- Parameter `C` gibt die <ins>inverse</ins> Stärke der Regularisierung an (Details siehe [hier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)).


1. Trainieren Sie eine Logistische Regression mit Lasso-Regression für verschiedene Parameter `C`:
    - den Default-Wert `C=1.0`.
    - wählen Sie `C` so, dass das Modell sehr stark regularisiert ist.
    - wählen Sie `C` so, dass das Modell keine Regularisierung hat.
2. Vergleiche Sie die Ergebnisse dieser Modelle indem Sie:
    - die Gewichte und deren Summe ausgeben (wie im Notebook [`4_OverUnderfitting.ipynb`](4_OverUnderfitting.ipynb)).
    - die Accuracy auf den Testdaten ermitteln. 

In [56]:
# train logistic regression model with lasso regularization using LogisticRegression
from sklearn.linear_model import LogisticRegression
# default C=1.0
lassoReg = LogisticRegression(max_iter=1000, penalty='l1', C=1.0, solver='liblinear')

lassoReg.fit(X_train, y_train)

pred = lassoReg.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, pred.round()))
print("Feature weights:")
print(pd.Series(lassoReg.coef_[0], index=X_train.columns))
print("Sum of absolute weights: ", np.sum(np.abs(lassoReg.coef_)))

# strong regularization (C=0.01)
lassoReg = LogisticRegression(max_iter=1000, penalty='l1', C=0.01, solver='liblinear')

lassoReg.fit(X_train, y_train)

pred = lassoReg.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, pred.round()))
print("Feature weights:")
print(pd.Series(lassoReg.coef_[0], index=X_train.columns))
print("Sum of absolute weights: ", np.sum(np.abs(lassoReg.coef_)))

# weak regularization (C=100)
lassoReg = LogisticRegression(max_iter=1000, penalty='l1', C=100, solver='liblinear')

lassoReg.fit(X_train, y_train)

pred = lassoReg.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, pred.round()))
print("Feature weights:")
print(pd.Series(lassoReg.coef_[0], index=X_train.columns))
print("Sum of absolute weights: ", np.sum(np.abs(lassoReg.coef_)))

Accuracy score:  0.6923076923076923
Feature weights:
Pclass   -0.936605
Age      -0.037646
SibSp    -0.308796
Parch     0.298062
Fare      0.007298
dtype: float64
Sum of absolute weights:  1.5884066329559943
Accuracy score:  0.6643356643356644
Feature weights:
Pclass    0.000000
Age      -0.028366
SibSp     0.000000
Parch     0.000000
Fare      0.016453
dtype: float64
Sum of absolute weights:  0.044819110820533595
Accuracy score:  0.6853146853146853
Feature weights:
Pclass   -1.051885
Age      -0.041738
SibSp    -0.336227
Parch     0.327037
Fare      0.005961
dtype: float64
Sum of absolute weights:  1.7628477678030783


### Aufgabe 4. Cross-Validierung über  `C`
Ein guter Wert für `C` soll nun mit Hilfe der Methode `cross_val_score` gefunden werden. Ein Beispiel für diese Methode ist:

In [57]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")
cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")

array([0.65217391, 0.73684211, 0.6754386 , 0.71929825, 0.71052632])

Die Methode führt eine k-fold Cross-Validation auf den Trainingsdaten durch und gibt für jeden der k-folds den Score (in diesem Fall "accuracy") des Modell auf den jeweiligen Validierungsdaten zurück.

1. Berechnen Sie den Durchschnitt der Rückgabewerte von `cross_val_score` um für das gegebene `model` den durchschnittlichen Accuarcy-Score über alle k-folds zu erhalten.
2. Berechnen Sie für jeden der Werte $ C \in [0.1, 0.2, ...,9.8, 9.9]$ den durchschnittlichen Accuarcy-Score.
3. Trainieren Sie das Modell mit dem besten Wert für $C$ auf allen Trainingsdaten.
4. Bestimmen Sie die Accuracy dieses Modells auf den Testdaten.

In [58]:
# create a list of C ranging from 0.1 to 9.9
Cs = np.arange(0.1, 10, 0.1)

# q: what is 0.1 and 10 in np.arange(0.1, 10, 0.1)?
# a: 0.1 and 10 are the start and end values, and 0.1 is the step size

print(f"Testing {len(Cs)} different values for C")

# create LogisticRegression model with l1 penalty and liblinear solver for each C
best_C, best_score = 0, 0
for C in Cs:
	model = LogisticRegression(max_iter=1000, penalty = "l1", C=C, solver="liblinear")
	scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
	
	# calculate mean of scores and compare to best score so far
	mean_score = np.mean(scores)
	if mean_score > best_score:
		best_score = mean_score
		best_C = C
	
print("Best C: ", best_C)
print("Best Score: ", best_score)

Testing 99 different values for C
Best C:  0.30000000000000004
Best Score:  0.7216475972540046


In [59]:
# train model with best C value
model = LogisticRegression(max_iter=1000, penalty = "l1", C=best_C, solver="liblinear")
model.fit(X_train, y_train)

# determine accuracy on the test set
pred = model.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, pred.round()))

Accuracy score:  0.6923076923076923
